<a href="https://colab.research.google.com/github/ryderwishart/biblical-machine-learning/blob/main/gpt-inferences/greek-hebrew-tsv-qa-agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question answer over MACULA Greek an Hebrew

In [1]:
# !pip install tabulate pandas langchain

## Set up MACULA dataframes

First, let's load up the Greek data and populate the domain column with the domain labels instead of numbers.

In [2]:
import requests, json, re, os
import pandas as pd

def download_file(url, file_name):
    response = requests.get(url)
    with open(file_name, "wb") as file:
        file.write(response.content)

# file1_url = "https://raw.githubusercontent.com/Clear-Bible/macula-greek/main/Nestle1904/TSV/macula-greek.tsv"
file1_url = "https://github.com/Clear-Bible/macula-greek/raw/feature/add-sentence-id-to-tsv/Nestle1904/TSV/macula-greek.tsv" # PR version with sentence IDs
file1_name = "macula-greek.tsv"

# file1_url = "https://raw.githubusercontent.com/Clear-Bible/macula-greek/main/Nestle1904/TSV/macula-greek.tsv"
file2_url = "https://raw.githubusercontent.com/Clear-Bible/macula-greek/main/sources/MARBLE/SDBG/marble-domain-label-mapping.json"
file2_name = "marble-domain-label-mapping.json"

if file1_name not in os.listdir():
    download_file(file1_url, file1_name)

if file2_name not in os.listdir():
    download_file(file2_url, file2_name)


In [3]:
# Import Macula Greek data
mg = pd.read_csv(
    "macula-greek.tsv", index_col="xml:id", sep="\t", header=0, converters={"*": str}
).fillna("missing")
# add an 'id' column
mg["id"] = mg.index

# mg['domain'] = mg['domain'].astype(str).fillna('missing')

# Extract book, chapter, and verse into separate columns
mg[["book", "chapter", "verse"]] = mg["ref"].str.extract(r"(\d?[A-Z]+)\s(\d+):(\d+)")

# Add columns for book + chapter, and book + chapter + verse for easier grouping
mg["book_chapter"] = mg["book"] + " " + mg["chapter"].astype(str)
mg["book_chapter_verse"] = mg["book_chapter"] + ":" + mg["verse"].astype(str)

# Import domain-label mapping
# Open the JSON file
with open("marble-domain-label-mapping.json", "r") as f:
    # Load the contents of the file as a dictionary
    domain_labels = json.load(f)

domain_labels["missing"] = "no domain"
domain_labels["nan"] = "no domain"

# Use domain labels to create a new column


def get_domain_label(domain_string_number):
    labels = [domain_labels[label] for label in domain_string_number.split(" ")]
    return labels


mg["domain_label"] = mg["domain"].apply(get_domain_label)
mg.head()

,ref,role,class,type,gloss,text,after,lemma,normalized,strong,...,frame,subjref,referent,id,book,chapter,verse,book_chapter,book_chapter_verse,domain_label
xml:id,,,,,,,,,,,,,,,,,,,,,
n40001001001,MAT 1:1!1,missing,noun,common,[The] book,Βίβλος,,βίβλος,Βίβλος,976,...,missing,missing,missing,n40001001001,MAT,1,1,MAT 1,MAT 1:1,[Written Language]
n40001001002,MAT 1:1!2,missing,noun,common,of [the] genealogy,γενέσεως,,γένεσις,γενέσεως,1078,...,missing,missing,missing,n40001001002,MAT,1,1,MAT 1,MAT 1:1,[Kinship Relations Involving Successive Genera...
n40001001003,MAT 1:1!3,missing,noun,proper,of Jesus,Ἰησοῦ,,Ἰησοῦς,Ἰησοῦ,2424,...,missing,missing,missing,n40001001003,MAT,1,1,MAT 1,MAT 1:1,[Persons]
n40001001004,MAT 1:1!4,missing,noun,proper,Christ,Χριστοῦ,,Χριστός,Χριστοῦ,5547,...,missing,missing,missing,n40001001004,MAT,1,1,MAT 1,MAT 1:1,[Persons]
n40001001005,MAT 1:1!5,missing,noun,common,son,υἱοῦ,,υἱός,υἱοῦ,5207,...,missing,missing,missing,n40001001005,MAT,1,1,MAT 1,MAT 1:1,[Kinship Relations Involving Successive Genera...


Second, let's load up the Hebrew data and populate the domain column with the domain labels instead of numbers.

In [4]:
# MACULA Hebrew URL: https://media.githubusercontent.com/media/Clear-Bible/macula-hebrew/main/TSV/macula-hebrew.tsv

file3_url = "https://raw.githubusercontent.com/Clear-Bible/macula-hebrew/main/TSV/macula-hebrew.tsv"
file3_name = "macula-hebrew.tsv"

file4_url = "https://raw.githubusercontent.com/Clear-Bible/macula-hebrew/main/sources/MARBLE/SDBH/domain-label-mapping-1.json"
file4_name = "domain-label-mapping-1.json"

if file3_name not in os.listdir():
    # This is a GIT LFS file, so we need to ensure the LFS protocol is used (which wget seems to resolve automatically)
    !wget https://media.githubusercontent.com/media/Clear-Bible/macula-hebrew/main/TSV/macula-hebrew.tsv
    
if file4_name not in os.listdir():
    download_file(file4_url, file4_name)

# Import Macula Hebrew data
mh = pd.read_csv(
    "macula-hebrew.tsv", index_col="xml:id", sep="\t", header=0, converters={"*": str}
).fillna("missing")
# add an 'id' column
mh["id"] = mh.index

# mh['domain'] = mh['domain'].astype(str).fillna('missing')

# Extract book, chapter, and verse into separate columns
mh[["book", "chapter", "verse"]] = mh["ref"].str.extract(r"(\d?[A-Z]+)\s(\d+):(\d+)")
# Add columns for book + chapter, and book + chapter + verse for easier grouping
mh["book_chapter"] = mh["book"] + " " + mh["chapter"].astype(str)
mh["book_chapter_verse"] = mh["book_chapter"] + ":" + mh["verse"].astype(str)

# Import domain-label mapping to Hebrew as well using SBDH domain mapping
# Open the JSON file
with open("domain-label-mapping-1.json", "r") as f:
    # Load the contents of the file as a dictionary
    hebrew_domain_labels = json.load(f)

def get_hebrew_domain_label(domain_string_number):
    labels = [hebrew_domain_labels[label] for label in domain_string_number.split(">") if label in hebrew_domain_labels]
    return labels

mh["domain_label"] = mh["lexdomain"].apply(get_hebrew_domain_label)

mh.head()

--2023-06-05 11:57:53--  https://media.githubusercontent.com/media/Clear-Bible/macula-hebrew/main/TSV/macula-hebrew.tsv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75034943 (72M) [text/plain]
Saving to: ‘macula-hebrew.tsv’

macula-hebrew.tsv   100%[===================>]  71.56M  6.77MB/s    in 13s     

2023-06-05 11:58:08 (5.61 MB/s) - ‘macula-hebrew.tsv’ saved [75034943/75034943]



/var/folders/hz/6rspy0v17q7b21wllgrjbn040000gn/T/ipykernel_10913/3928210015.py:17: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  mh = pd.read_csv(


,ref,class,text,transliteration,after,strongnumberx,stronglemma,sensenumber,greek,greekstrong,...,frame,subjref,participantref,id,book,chapter,verse,book_chapter,book_chapter_verse,domain_label
xml:id,,,,,,,,,,,,,,,,,,,,,
o010010010011,GEN 1:1!1,prep,בְּ,bə,missing,0871a,b,missing,ἐν,1722.0,...,missing,missing,missing,o010010010011,GEN,1,1,GEN 1,GEN 1:1,[]
o010010010012,GEN 1:1!1,noun,רֵאשִׁ֖ית,rēʾšiyṯ,,7225,7225,1.0,ἀρξῇ,746.0,...,missing,missing,missing,o010010010012,GEN,1,1,GEN 1,GEN 1:1,[Begin]
o010010010021,GEN 1:1!2,verb,בָּרָ֣א,bārāʾ,,1254,1254 a,1.0,ἐποίησεν,4160.0,...,A0:010010010031; A1:010010010052;010010010072;,missing,missing,o010010010021,GEN,1,1,GEN 1,GEN 1:1,[Exist]
o010010010031,GEN 1:1!3,noun,אֱלֹהִ֑ים,ʾĕlōhiym,,0430,430,1.0,θεὸς,2316.0,...,missing,missing,missing,o010010010031,GEN,1,1,GEN 1,GEN 1:1,[Deities]
o010010010041,GEN 1:1!4,om,אֵ֥ת,ʾēṯ,,0853,853,missing,missing,missing,...,missing,missing,missing,o010010010041,GEN,1,1,GEN 1,GEN 1:1,[Identifiers]


In [5]:
print("Macula Greek column names: ", mg.columns)
print("Macula Hebrew column names: ", mh.columns)

Macula Greek column names:  Index(['ref', 'role', 'class', 'type', 'gloss', 'text', 'after', 'lemma',
       'normalized', 'strong', 'morph', 'person', 'number', 'gender', 'case',
       'tense', 'voice', 'mood', 'degree', 'domain', 'ln', 'frame', 'subjref',
       'referent', 'id', 'book', 'chapter', 'verse', 'book_chapter',
       'book_chapter_verse', 'domain_label'],
      dtype='object')
Macula Hebrew column names:  Index(['ref', 'class', 'text', 'transliteration', 'after', 'strongnumberx',
       'stronglemma', 'sensenumber', 'greek', 'greekstrong', 'gloss',
       'english', 'mandarin', 'stem', 'morph', 'lang', 'lemma', 'pos',
       'person', 'gender', 'number', 'state', 'type', 'lexdomain',
       'contextualdomain', 'coredomain', 'sdbh', 'extends', 'frame', 'subjref',
       'participantref', 'id', 'book', 'chapter', 'verse', 'book_chapter',
       'book_chapter_verse', 'domain_label'],
      dtype='object')


## Set up QA agent

In [6]:
import getpass
secret_key = getpass.getpass('Enter OpenAI secret key: ')
os.environ['OPENAI_API_KEY'] = secret_key

In [7]:
import pandas as pd
from langchain.llms import OpenAI
from langchain.agents import create_pandas_dataframe_agent

In [8]:
multi_df_agent = create_pandas_dataframe_agent(
    OpenAI(temperature=0), 
    [mg, mh], # list of dataframes, here mg = Macula Greek, mh = Macula Hebrew
    verbose=True,
    max_tokens_limit = 3000,
    max_tokens_per_prompt = 1000,
)

In [9]:
# multi_df_agent.run("How many unique domain labels are there (each domain label is a list)?")

In [10]:
multi_df_agent.run("If you group all the rows by book, which books do not have some form of 'lord' or 'Lord' in their aggregated gloss column?")

This was a bit shocking to me, but neither First, Second, or Third John, or Titus contain the word 'Lord'! To double check this, look at the vocab list through [Scaife](https://vocab.perseus.org/word-list/urn:cts:greekLit:tlg0031.tlg017.perseus-grc2/?page=all)

## Additional functionality

### List building

Now let's try to generate a list of results and output as TSV or plaintext.

In [15]:
macula_greek_agent = create_pandas_dataframe_agent(
    OpenAI(temperature=0), 
    mg, # only one dataframe
    verbose=True,
    max_tokens_limit = 3000,
    max_tokens_per_prompt = 1000,
)

We have to be pretty specific in the instructions we give the model for now, since it has to be able to understand what we might want to do with a Pandas dataframe. Thus, the following prompt breaks down a couple of steps:
1. Group the data in some way
2. Filter by criterion A
3. Filter again by criterion B

In [19]:
list_building_prompt = """If you group the dataframe by book_chapter_verse, \
list all the verses that mention 'Jesus', and then filter by all the verses \
that also mention 'Peter' in their aggregated gloss column. \
"""

macula_greek_agent.run(list_building_prompt)



> Entering new AgentExecutor chain...
Thought: I need to group the dataframes by book_chapter_verse, then filter by the gloss column.
Action: python_repl_ast
Action Input: df1.groupby('book_chapter_verse')['gloss'].agg(lambda x: ' '.join(x)).reset_index().loc[df1.groupby('book_chapter_verse')['gloss'].agg(lambda x: ' '.join(x)).reset_index()['gloss'].str.contains('Jesus') & df1.groupby('book_chapter_verse')['gloss'].agg(lambda x: ' '.join(x)).reset_index()['gloss'].str.contains('Peter')]
Observation:      book_chapter_verse                                              gloss
542             1PE 1:1  Peter an apostle of Jesus Christ To [the] elec...
1118            2PE 1:1  Simon Peter a servant and apostle of Jesus Chr...
2048           ACT 2:38  Peter then to them Repent and be baptized ever...
2086            ACT 3:6  Said however Peter Silver and gold none there ...
2094           ACT 4:13  Seeing now the - of Peter boldness and of John...
2310           ACT 9:34  And said to him -

"The verses that mention both 'Jesus' and 'Peter' are 1PE 1:1, 2PE 1:1, ACT 2:38, ACT 3:6, ACT 4:13, ACT 9:34, JHN 13:36, JHN 13:8, JHN 18:11, JHN 18:15, JHN 1:42, JHN 20:2, JHN 21:15, JHN 21:17, JHN 21:20, JHN 21:21, JHN 21:7, LUK 5:8, LUK 8:45, LUK 9:33, MAT 14:29, MAT 17:1, MAT 17:4, MAT 26:69, MAT 26:75, MAT 8:14, MRK 14:67, MRK 14:72, MRK 9:2, MRK 9:5."

Great, so we were able to get the Agent to list out the findings, without actually having to write Python code to manipulate the Pandas dataframe ourselves!

Now let's try adding an output formatting directive to the prompt, namely, 
- Output as CSV
- Use a particular set of headings

In [24]:
list_building_and_save_as_csv_prompt = list_building_prompt + """\
Please return the results in a CSV with the following columns: \
'book_chapter_verse', 'gloss'. \
"""

macula_greek_agent.run(list_building_and_save_as_csv_prompt)



> Entering new AgentExecutor chain...

Thought: I need to group the dataframes by book_chapter_verse, then filter by the words 'Jesus' and 'Peter' in the gloss column.
Action: python_repl_ast
Action Input: df1.groupby('book_chapter_verse')['gloss'].agg(lambda x: ' '.join(x)).reset_index().loc[(df1['gloss'].str.contains('Jesus')) & (df1['gloss'].str.contains('Peter'))].to_csv('results.csv', index=False)
Observation: IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).
Thought: I need to make sure the dataframes are aligned before I can filter.
Action: python_repl_ast
Action Input: df1.set_index('book_chapter_verse', inplace=True)
df2.set_index('book_chapter_verse', inplace=True)
Observation: 
Thought: Now I can filter by the words 'Jesus' and 'Peter' in the gloss column.
Action: python_repl_ast
Action Input: df1.groupby('book_chapter_verse')['gloss'].agg(lambda x: ' '.join(x)).reset_index().loc[(df1['gloss'

"The final answer is a CSV file with the columns 'book_chapter_verse', 'gloss' containing all the verses that mention 'Jesus' and 'Peter' in their aggregated gloss column."

Check the folder where this notebook is stored, and you should actually see a file called `result.csv` or `results.csv`, which contains the data we just extracted from the original dataframes, but we only had to query in natural language!

In [36]:
import gradio as gr

def qa_run(inputs):
    return multi_df_agent.run(inputs)

def lb_run(inputs):
    return macula_greek_agent.run(inputs)

def csv_run(inputs):
    return macula_greek_agent.run(inputs)

app = gr.Blocks(theme='bethecloud/storj_theme')

with app:
    gr.Markdown("## LangChain")
    gr.Markdown("A tool for querying and manipulating dataframes with OpenAI's GPT-3.")
    gr.Markdown("[Read More](https://medium.com/@langchain/using-openai-gpt-3-to-query-and-manipulate-dataframes-1f4f8f211e4d)")

    with gr.Tab("Question Answering"):
        qa_text = gr.Textbox(lines=5, label="Question", value="How many unique domain labels are there (each domain label is a list)?")
        qa_button = gr.Button("Run")
        qa_output = gr.Textbox(label="Answer", lines=5)
        qa_button.click(
            qa_run,
            inputs=qa_text,
            outputs=qa_output,
        )
    with gr.Tab("List Building"):
        lb_text = gr.Textbox(lines=5, label="Prompt", value=list_building_prompt)
        lb_button = gr.Button("Run")
        lb_output = gr.Textbox(label="List", lines=5)
        lb_button.click(
            lb_run,
            inputs=lb_text,
            outputs=lb_output,
        )
    with gr.Tab("CSV Generation"):
        csv_text = gr.Textbox(lines=5, label="Prompt", value=list_building_and_save_as_csv_prompt)
        csv_button = gr.Button("Run")
        csv_output = gr.Textbox(label="CSV", lines=5)
        csv_button.click(
            csv_run,
            inputs=csv_text,
            outputs=csv_output,
        )

app.launch(debug=True)



Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
